<a href="https://colab.research.google.com/github/Mariannly/Quark5-CoAfina2025/blob/main/leer_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "cdsapi>=0.7.7"
!pip install xarray
!pip install cfgrib

In [2]:
import os
import cdsapi
import time
import xarray as xr
import cfgrib

In [3]:
cdsapirc_content = """url: https://cds.climate.copernicus.eu/api
key: 01876cbb-e8a2-4cb4-8d98-5f4ba769fc1a
"""

# Crear el archivo de configuración en la ruta esperada
with open(os.path.expanduser("~/.cdsapirc"), "w") as f:
    f.write(cdsapirc_content)

print("✅ Archivo .cdsapirc creado correctamente")


✅ Archivo .cdsapirc creado correctamente


In [4]:
c = cdsapi.Client()
print("✅ Conexión establecida con Copernicus CDS API")

✅ Conexión establecida con Copernicus CDS API


In [5]:
dataset = "reanalysis-era5-land-monthly-means"
bbox_riohacha = {
    "north": 11.80,
    "south": 11.30,
    "west": -73.20,
    "east": -72.60,
}
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": [
        "total_precipitation",
        "2m_temperature",
        "volumetric_soil_water_layer_1",
        "volumetric_soil_water_layer_2",
        "volumetric_soil_water_layer_3",
        "volumetric_soil_water_layer_4",
        "surface_solar_radiation_downwards",
        "evaporation_from_vegetation_transpiration",
        "evaporation_from_bare_soil",
    ],
    "year": [str(y) for y in range(1985, 2025)],   # 1985–2024
    "month": [f"{m:02d}" for m in range(1, 13)],
    "time": "00:00",
    "area": [
        bbox_riohacha["north"],
        bbox_riohacha["west"],
        bbox_riohacha["south"],
        bbox_riohacha["east"],
    ],
    "format": "netcdf",
}

c.retrieve(dataset, request).download()


2025-11-08 00:25:32,247 INFO Request ID is 4865cc46-a7d9-43ef-b85f-82b9e5ba23a3
INFO:ecmwf.datastores.legacy_client:Request ID is 4865cc46-a7d9-43ef-b85f-82b9e5ba23a3
2025-11-08 00:25:32,493 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-08 00:25:41,703 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-11-08 00:25:47,003 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


d10a2162d2759060cd3b0a8c1c25d670.zip:   0%|          | 0.00/319k [00:00<?, ?B/s]

'd10a2162d2759060cd3b0a8c1c25d670.zip'

In [6]:
data_file = 'd10a2162d2759060cd3b0a8c1c25d670.zip'
#umcompress file
!unzip -q {data_file}

replace data_stream-moda.nc? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [7]:


# Open the GRIB file using xarray
ds = xr.open_dataset(data_file, engine='cfgrib')

# Display the dataset information
print(ds)

Streaming output truncated to the last 5000 lines.
  File "/usr/local/lib/python3.12/dist-packages/cfgrib/messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/cfgrib/messages.py", line 97, in from_file
    codes_id = eccodes.codes_grib_new_from_file(file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gribapi/gribapi.py", line 415, in grib_new_from_file
    GRIB_CHECK(err)
  File "/usr/local/lib/python3.12/dist-packages/gribapi/gribapi.py", line 232, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "/usr/local/lib/python3.12/dist-packages/gribapi/errors.py", line 386, in raise_grib_error
    raise ERROR_MAP[errid](errid)
gribapi.errors.UnsupportedEditionError: Edition not supported.
ERROR:cfgrib.messages:skipping corrupted Message
Traceback (most recent call last):
  File "/usr/loc

EOFError: No valid message found: 'd10a2162d2759060cd3b0a8c1c25d670.zip'